In [414]:
import pandas as pd
import numpy as np

In [415]:
formerCast_df = pd.read_csv('formerCast.csv')
currentCast_df = pd.read_csv('currentCast.csv')
specialCast_df = pd.read_csv('specialCast.csv')

In [416]:
fc_episode = pd.read_csv('fc_episodes.csv')
sc_episode = pd.read_csv('sc_episodes.csv')
cc_episode = pd.read_csv('cc_episodes.csv')

In [417]:
fc_episode = fc_episode[fc_episode['season'] > 0]
sc_episode = sc_episode[sc_episode['season'] > 0]
cc_episode = cc_episode[cc_episode['season'] > 0]

In [418]:
fc_episode = fc_episode.reset_index(drop=True)
sc_episode = sc_episode.reset_index(drop=True)
cc_episode = cc_episode.reset_index(drop=True)

In [419]:
fc_episode['startEp'] = fc_episode['startEp'].astype('int')
fc_episode['endEp'] = fc_episode['endEp'].astype('int')
fc_episode = fc_episode.set_index('father')

sc_episode['startEp'] = sc_episode['startEp'].astype('int')
sc_episode['endEp'] = sc_episode['endEp'].astype('int')
sc_episode = sc_episode.set_index('father')

cc_episode['startEp'] = cc_episode['startEp'].astype('int')
cc_episode['endEp'] = cc_episode['endEp'].astype('int')
cc_episode = cc_episode.set_index('father')

In [420]:
def encoding_episodes(row):
    row.loc[row['startEp']-1 + (4): row['endEp']+ (4)] = str(row['dad'])
    return row

def get_table(episode_df):
    #start_df = pd.get_dummies(episode_df['startEp'])
    start_df = episode_df.copy()
    i = 1
    while i <= 304:
        start_df[i] = None
        i = i + 1
    #start_df['totalNoOfEp'] = fc_episode['totalNoOfEp']
    #
    start_df['dad'] = episode_df.index
    
    
    start_df = start_df.apply(encoding_episodes, axis=1)
    #start_df = pd.pivot_table(start_df, values=intList, columns=['father'], aggfunc='max')
    start_df = pd.pivot_table(start_df, values=intList, columns=['father'], aggfunc = lambda x: x.any() if x.notnull().values.any() else None)
    return start_df

In [421]:
fc_encoded_df = get_table(fc_episode)
sc_encoded_df = get_table(sc_episode)
cc_encoded_df = get_table(cc_episode)

In [281]:
'''
sampleDF = pd.DataFrame({'dad': ['Dad A', 'Dad B', 'Dad C'],
                        '1': [1, 0, 0],
                        '2': [1, 1, 0],
                        '3': [0, 0, 1]})
'''

In [282]:
#pd.pivot_table(sampleDF, values=['1', '2', '3'], columns=['dad'])

dad,Dad A,Dad B,Dad C
1,1,0,0
2,1,1,0
3,0,0,1


In [325]:
#x = sampleDF.stack()
#x.index = x.index.droplevel()
#x
#x.drop(['dad'], axis='rows')

In [272]:
#fc_encoded_df['episode'] = fc_encoded_df.index
#sc_encoded_df['episode'] = sc_encoded_df.index
#cc_encoded_df['episode'] = cc_encoded_df.index

In [446]:
fc_encoded_df = pd.DataFrame(fc_encoded_df.stack())
fc_encoded_df.columns = ['father']
fc_encoded_df.index = fc_encoded_df.index.droplevel(1)

sc_encoded_df = pd.DataFrame(sc_encoded_df.stack())
sc_encoded_df.columns = ['father']
sc_encoded_df.index = sc_encoded_df.index.droplevel(1)

cc_encoded_df = pd.DataFrame(cc_encoded_df.stack())
cc_encoded_df.columns = ['father']
cc_encoded_df.index = cc_encoded_df.index.droplevel(1)

In [451]:
total_encoded_df = pd.concat([fc_encoded_df,sc_encoded_df,cc_encoded_df])

In [447]:
import os
directory = 'data'
if not os.path.exists(directory):
    os.makedirs(directory)

In [452]:
dict_df = {'fc_encoded.csv': fc_encoded_df, 
           'sc_encoded.csv': sc_encoded_df, 
           'cc_encoded.csv': cc_encoded_df,
            'total_encoded.csv': total_encoded_df}

for key,val in dict_df.items():
    path = os.path.join(directory, key)
    val.to_csv(path, encoding='utf-8')